In [1]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:

code = """
#include <stdio.h> 
#include <stdlib.h>
#define N 1000000
__global__ void add(int *a, int *b, int *c) {
int tid = blockIdx.x * blockDim.x + threadIdx.x; if (tid < N) {
c[tid] = a[tid] + b[tid]; }
}
int main() {
int *a, *b, *c;
int *d_a, *d_b, *d_c;
int size = N * sizeof(int);

// Allocate memory on host 
a = (int*)malloc(size);
b = (int*)malloc(size);
c = (int*)malloc(size);

// Initialize arrays
for (int i = 0; i < N; i++) {
a[i] = i;
b[i] = i * 2; }

// Allocate memory on device 
cudaMalloc(&d_a, size); 
cudaMalloc(&d_b, size); 
cudaMalloc(&d_c, size);

// Copy data from host to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice); 
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

// Launch kernel with 1 million threads 
add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);

// Copy result from device to host
cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

// Print first and last elements of result 
printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);

// Free memory 
free(a); 
free(b); 
free(c); 
cudaFree(d_a);
cudaFree(d_b); 
cudaFree(d_c);
return 0; }
"""


In [4]:
text_file = open("assign4.cu","w") 
text_file.write(code) 
text_file.close()

In [5]:
!nvcc assign4.cu


In [6]:
!./a.out

c[0]=0, c[999999] = 2999997

In [7]:
!nvprof ./a.out

==983== NVPROF is profiling process 983, command: ./a.out
==983== c[0]=0, c[999999] = 2999997Profiling application: ./a.out
==983== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.81%  1.4915ms         1  1.4915ms  1.4915ms  1.4915ms  [CUDA memcpy DtoH]
                   47.58%  1.3964ms         2  698.22us  668.11us  728.34us  [CUDA memcpy HtoD]
                    1.61%  47.327us         1  47.327us  47.327us  47.327us  add(int*, int*, int*)
      API calls:   97.19%  210.05ms         3  70.018ms  62.203us  209.92ms  cudaMalloc
                    2.10%  4.5352ms         3  1.5117ms  894.31us  2.7241ms  cudaMemcpy
                    0.39%  845.06us         1  845.06us  845.06us  845.06us  cuDeviceGetPCIBusId
                    0.24%  513.04us         3  171.01us  110.16us  202.44us  cudaFree
                    0.05%  111.67us       101  1.1050us     128ns  47.162us  cuDeviceGetAttribute
                    